In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.GraphUtils import GraphUtils

In [7]:
# Generate data
X = np.random.uniform(size=1000)
eps = np.random.normal(size=1000)
delta = np.random.uniform(size=1000)
Z = -7*X + 0.5*delta
Y = 2*X + Z + eps

# Create DataFrame with named variables
data = pd.DataFrame({'X': X, 'Y': Y, 'Z': Z})

In [8]:
# Store original column names
variable_names = list(data.columns)

# Run FCI
g, edge = fci(data.to_numpy(), alpha=0.2)

# Get the dot representation
dot = GraphUtils.to_pydot(g)

# Relabel the nodes with original variable names
for i, node in enumerate(dot.get_nodes()):
    if node.get_name().isdigit() and int(node.get_name()) < len(variable_names):
        # Replace the node name with the original variable name
        node.set_name(f'"{variable_names[int(node.get_name())]}"')

# Save the labeled graph
dot.write_png('labeled_graph.png')

# Print the mapping for reference
print("Variable mapping:")
for i, name in enumerate(variable_names):
    print(f"Node {i} -> {name}")

Depth=0, working on node 2: 100%|██████████| 3/3 [00:00<00:00, 563.20it/s]


Variable mapping:
Node 0 -> X
Node 1 -> Y
Node 2 -> Z
